In [1]:
%matplotlib inline

import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from prettytable import PrettyTable
from IPython.display import Image

from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Add, MaxPooling1D, BatchNormalization
from keras.layers import Embedding, Bidirectional, CuDNNLSTM, GlobalMaxPooling1D

2023-01-19 08:15:42.511700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Text Preprocessing

In [2]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1

  return char_dict

char_dict = create_dict(codes)

print(char_dict)
print("Dict Length:", len(char_dict))

{'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
Dict Length: 20


In [3]:
dataset = pd.read_csv("../Fhalab/virus.csv")
df = dataset[['FASTA_com', 'IC50']].copy()
df.loc[df['IC50'] <= 10, 'IC50'] = 1
df.loc[df['IC50'] > 10, 'IC50'] = 0

# df.loc[df['IC50'] <= 10, 'IC50'] = 'YES'
# df.loc[df['IC50'] > 10, 'IC50'] = 'NO'
df = df.dropna()

BLOSUM

In [5]:
train_df = df.sample(frac=0.8, random_state=0)
label_train = train_df['IC50']
train_df = train_df.drop('IC50',axis=1)
test_df = df.drop(train_df.index)
label_test = test_df['IC50']
test_df = test_df.drop('IC50',axis=1)

In [43]:
import seaborn as sns
import epitopepredict as ep
blosum = ep.blosum62

def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def blosum_encode(seq):
    #encode a peptide into blosum features
    s=list(seq)
    x = pd.DataFrame([blosum[i] for i in seq]).reset_index(drop=True)
    # show_matrix(x)
    e = x.values.flatten()    
    return e

pep='ALDFEQEMT'
e=blosum_encode(pep)
e.shape

(216,)

In [44]:
train_ohe = train_df['FASTA_com'].apply(lambda x: pd.Series(blosum_encode(x)),1)
test_ohe = test_df['FASTA_com'].apply(lambda x: pd.Series(blosum_encode(x)),1)

In [45]:
train_ohe.fillna(0, inplace = True)
test_ohe.fillna(0, inplace = True)


In [46]:
train_pad = train_ohe
test_pad = test_ohe


ONE-HOT

In [6]:
def integer_encoding(data):
  """
  - Encodes code sequence to integer values.
  - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
  """
  
  encode_list = []
  for row in data['FASTA_com'].values:
    row_encode = []
    for code in row:
      row_encode.append(char_dict.get(code, 0))
    encode_list.append(np.array(row_encode))
  
  return encode_list


train_encode = integer_encoding(train_df) 
# val_encode = integer_encoding(val_sm) 
test_encode = integer_encoding(test_df) 

In [7]:
# padding sequences
max_length = 400
train_pad = pad_sequences(train_encode, maxlen=max_length, padding='post', truncating='post')
# val_pad = pad_sequences(val_encode, maxlen=max_length, padding='post', truncating='post')
test_pad = pad_sequences(test_encode, maxlen=max_length, padding='post', truncating='post')

train_pad.shape, test_pad.shape

((1465, 400), (366, 400))

In [8]:
train_ohe = train_pad
test_ohe = test_pad

In [47]:
from keras.utils import to_categorical

# One hot encoding of sequences
train_ohe = to_categorical(train_pad)
# val_ohe = to_categorical(val_pad)
test_ohe = to_categorical(test_pad)

train_ohe.shape, test_ohe.shape

((1465, 1176, 12), (366, 1176, 12))

In [8]:
label_train.shape, label_test.shape

((1465,), (366,))

In [9]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [45]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

#   val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
#   print('Val loss: ', val_score[0])
#   print('Val accuracy: ', val_score[1])
#   print('-'*70)
  
  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

Model 1: Bidirectional LSTM


In [11]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import TimeDistributed


In [69]:
x_input = Input(shape=(1176,))
emb = Embedding(20, 21, input_length=max_length)(x_input)
bi_rnn = Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(emb)
x = Dropout(0.3)(bi_rnn)

# softmax classifier
x_output = Dense(1, activation='sigmoid')(x)
# model1 = Sequential()
model1 = Model(inputs=x_input, outputs=x_output)
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

NameError: name 'GRU' is not defined

In [67]:
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)


In [68]:
history1 = model1.fit(
    train_pad, label_train,
    epochs=500, batch_size=100,
    validation_data=(test_pad, label_test),
    callbacks=[es]
    )

Epoch 1/500
15/15 [==============================] - 5s 176ms/step - loss: 5.8808 - accuracy: 0.6464 - val_loss: 5.1180 - val_accuracy: 0.6448
Epoch 2/500
15/15 [==============================] - 2s 146ms/step - loss: 4.6542 - accuracy: 0.6498 - val_loss: 4.1911 - val_accuracy: 0.6448
Epoch 3/500
15/15 [==============================] - 2s 144ms/step - loss: 3.9088 - accuracy: 0.6498 - val_loss: 3.6273 - val_accuracy: 0.6448
Epoch 4/500
15/15 [==============================] - 2s 146ms/step - loss: 3.4519 - accuracy: 0.6498 - val_loss: 3.2761 - val_accuracy: 0.6448
Epoch 5/500
15/15 [==============================] - 2s 145ms/step - loss: 3.1621 - accuracy: 0.6498 - val_loss: 3.0467 - val_accuracy: 0.6448
Epoch 6/500
15/15 [==============================] - 2s 145ms/step - loss: 2.9683 - accuracy: 0.6498 - val_loss: 2.8866 - val_accuracy: 0.6448
Epoch 7/500
15/15 [==============================] - 2s 140ms/step - loss: 2.8271 - accuracy: 0.6498 - val_loss: 2.7661 - val_accuracy: 0.6448

KeyboardInterrupt: 

In [40]:
plot_history(history1)

KeyError: 'acc'

ProtCNN

In [15]:
def residual_block(data, filters, d_rate):
  """
  _data: input
  _filters: convolution filters
  _d_rate: dilation rate
  """

  shortcut = data

  bn1 = BatchNormalization()(data)
  act1 = Activation('relu')(bn1)
  conv1 = Conv1D(filters, 1, dilation_rate=d_rate, padding='same', kernel_regularizer=l2(0.001))(act1)

  #bottleneck convolution
  bn2 = BatchNormalization()(conv1)
  act2 = Activation('relu')(bn2)
  conv2 = Conv1D(filters, 3, padding='same', kernel_regularizer=l2(0.001))(act2)

  #skip connection
  x = Add()([conv2, shortcut])

  return x

In [78]:
# model

x_input = Input(shape=(1176,1))

#initial conv
conv = Conv1D(128, 1, padding='same')(x_input) 

# per-residue representation
res1 = residual_block(conv, 128, 2)
res2 = residual_block(res1, 128, 3)

x = MaxPooling1D(3)(res2)
x = Dropout(0.5)(x)

# softmax classifier
x = Flatten()(x)
x_output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0001))(x)
model2= Sequential()
model2 = Model(inputs=x_input, outputs=x_output)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 1176, 1)]    0           []                               
                                                                                                  
 conv1d_27 (Conv1D)             (None, 1176, 128)    256         ['input_15[0][0]']               
                                                                                                  
 batch_normalization_20 (BatchN  (None, 1176, 128)   512         ['conv1d_27[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_20 (Activation)     (None, 1176, 128)    0           ['batch_normalization_20[0

In [79]:
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)


In [80]:
history2 = model2.fit(
    train_ohe, label_train,
    epochs=500, batch_size=100,
    validation_data=(test_ohe, label_test)#,
    #callbacks=[es]
    )

Epoch 1/500
15/15 [==============================] - 1s 34ms/step - loss: 3.2185 - accuracy: 0.6430 - val_loss: 1.4165 - val_accuracy: 0.6448
Epoch 2/500
15/15 [==============================] - 0s 19ms/step - loss: 1.2684 - accuracy: 0.7611 - val_loss: 1.1562 - val_accuracy: 0.6475
Epoch 3/500
15/15 [==============================] - 0s 19ms/step - loss: 0.9964 - accuracy: 0.7993 - val_loss: 1.0690 - val_accuracy: 0.7732
Epoch 4/500
15/15 [==============================] - 0s 19ms/step - loss: 0.9531 - accuracy: 0.8007 - val_loss: 1.0031 - val_accuracy: 0.8142
Epoch 5/500
15/15 [==============================] - 0s 19ms/step - loss: 0.9409 - accuracy: 0.8232 - val_loss: 0.9400 - val_accuracy: 0.8251
Epoch 6/500
15/15 [==============================] - 0s 19ms/step - loss: 0.9345 - accuracy: 0.8130 - val_loss: 0.9121 - val_accuracy: 0.8361
Epoch 7/500
15/15 [==============================] - 0s 19ms/step - loss: 0.9344 - accuracy: 0.8212 - val_loss: 0.9054 - val_accuracy: 0.8306
Epoch 

GRU

In [77]:
from keras.layers import Input, Embedding, GRU, Dropout, Dense
from keras.models import Model

# Define the maximum length of the input sequences
max_length = 1176

# Input layer
x_input = Input(shape=(max_length,))

# Embedding layer
emb = Embedding(20, 64, input_length=max_length)(x_input)

# GRU layer
gru_rnn = GRU(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(emb)

# Dropout layer
x = Dropout(0.3)(gru_rnn)

# Output layer
x_output = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=x_input, outputs=x_output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 1176)]            0         
                                                                 
 embedding_23 (Embedding)    (None, 1176, 64)          1280      
                                                                 
 gru_4 (GRU)                 (None, 64)                24960     
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 26,305
Trainable params: 26,305
Non-trainable params: 0
_________________________________________________________________


In [78]:
history1 = model.fit(
    train_pad, label_train,
    epochs=500, batch_size=100,
    validation_data=(test_pad, label_test),
    callbacks=[es]
    )

Epoch 1/500
15/15 [==============================] - 2s 72ms/step - loss: 2.0609 - accuracy: 0.6403 - val_loss: 1.8199 - val_accuracy: 0.6448
Epoch 2/500
15/15 [==============================] - 1s 53ms/step - loss: 1.6562 - accuracy: 0.6498 - val_loss: 1.4855 - val_accuracy: 0.6448
Epoch 3/500
15/15 [==============================] - 1s 53ms/step - loss: 1.3650 - accuracy: 0.6498 - val_loss: 1.2427 - val_accuracy: 0.6448
Epoch 4/500
15/15 [==============================] - 1s 52ms/step - loss: 1.1545 - accuracy: 0.6498 - val_loss: 1.0682 - val_accuracy: 0.6448
Epoch 5/500
15/15 [==============================] - 1s 52ms/step - loss: 1.0044 - accuracy: 0.6498 - val_loss: 0.9437 - val_accuracy: 0.6448
Epoch 6/500
15/15 [==============================] - 1s 52ms/step - loss: 0.8992 - accuracy: 0.6498 - val_loss: 0.8547 - val_accuracy: 0.6448
Epoch 7/500
15/15 [==============================] - 1s 54ms/step - loss: 0.8213 - accuracy: 0.6498 - val_loss: 0.7915 - val_accuracy: 0.6448
Epoch 